# CityBikes

Send a request to CityBikes for the city of your choice. 

In [2]:
#importing required libraries and packages
import requests
import json
import pandas as pd
import os

In [3]:
#find all bike stations for mobibikes company in Vancouver canada
request_url = f'http://api.citybik.es/v2/networks/mobibikes'
response=requests.get(request_url)
print(response)
data=response.json()
print(data)

<Response [200]>
{'network': {'company': ['Vanncouver Bike Share Inc.', 'CycleHop LLC', 'City of Vancouver', 'Shaw Communications Inc.', 'Fifteen'], 'gbfs_href': 'https://vancouver-gbfs.smoove.pro/gbfs/2/gbfs.json', 'href': '/v2/networks/mobibikes', 'id': 'mobibikes', 'location': {'city': 'Vancouver', 'country': 'CA', 'latitude': 49.2827, 'longitude': -123.1207}, 'name': 'Mobi', 'stations': [{'empty_slots': 13, 'extra': {'ebikes': 4, 'has_ebikes': True, 'last_updated': 1688487668, 'normal_bikes': 17, 'renting': True, 'returning': True, 'slots': 35, 'uid': '0001'}, 'free_bikes': 21, 'id': '7a19c49f486d7c0c02b3685d7b240448', 'latitude': 49.262487, 'longitude': -123.114397, 'name': '10th & Cambie', 'timestamp': '2023-07-04T16:25:07.793000Z'}, {'empty_slots': 1, 'extra': {'ebikes': 1, 'has_ebikes': True, 'last_updated': 1688487886, 'normal_bikes': 14, 'renting': True, 'returning': True, 'slots': 16, 'uid': '0004'}, 'free_bikes': 15, 'id': '32603a87cfca71d0f7dfa3513bad69d5', 'latitude': 49.

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [4]:
#create a dataframe to parse the result
stations = data['network']['stations']
bike_station_df = pd.json_normalize(data = stations)
print(bike_station_df.shape)
print(bike_station_df['free_bikes'].sum())
print(bike_station_df['empty_slots'].sum())
bike_station_df


(243, 15)
1980
2450


,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.ebikes,extra.has_ebikes,extra.last_updated,extra.normal_bikes,extra.renting,extra.returning,extra.slots,extra.uid
0,13,21,7a19c49f486d7c0c02b3685d7b240448,49.262487,-123.114397,10th & Cambie,2023-07-04T16:25:07.793000Z,4,True,1688487668,17,True,True,35,0001
1,1,15,32603a87cfca71d0f7dfa3513bad69d5,49.274566,-123.121817,Yaletown-Roundhouse Station,2023-07-04T16:25:07.790000Z,1,True,1688487886,14,True,True,16,0004
2,14,11,6d42fa40360f9a6b2bf641c7b8bb2862,49.279764,-123.110154,Dunsmuir & Beatty,2023-07-04T16:25:07.790000Z,1,True,1688487574,10,True,True,26,0005
3,9,6,66f873d641d448bd1572ab086665a458,49.260599,-123.113504,12th & Yukon (City Hall),2023-07-04T16:25:07.792000Z,5,True,1688487817,1,True,True,16,0007
4,5,10,485d4d24c803cfde829ab89699fed833,49.264215,-123.117772,8th & Ash,2023-07-04T16:25:07.799000Z,1,True,1688487829,9,True,True,16,0008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,23,0,cc2084e365f6f230fe26bae0b6867775,49.277178,-123.130000,Hornby & Drake,2023-07-04T16:25:07.718000Z,0,True,1688487801,0,True,True,24,0196
239,6,10,f38bac9712f3beee2149ba2a97048878,49.291597,-123.129158,Cardero & Bayshore,2023-07-04T16:25:07.660000Z,1,True,1688487545,9,True,True,20,0200
240,22,0,9ee3bc9e836768e3e04e4f3328dae03b,49.247204,-123.101549,27th & Main,2023-07-04T16:25:07.835000Z,0,True,1688487783,0,True,True,22,0565
241,13,4,aa1cbf97abc3cd50515bc54633a9bb2a,49.285670,-123.112543,Cordova & Granville,2023-07-04T16:25:07.841000Z,2,True,1688487889,2,True,True,18,0021


Put your parsed results into a DataFrame.

In [5]:
#dropping the rows where latitute, longitude have missing values
bike_station_df.dropna(subset=['latitude', 'longitude'], inplace=True)
bike_station_df['lat_long'] = bike_station_df['latitude'].astype(str) + ',' + bike_station_df['longitude'].astype(str)
bike_station_df = bike_station_df[bike_station_df['lat_long'] != '0.0,0.0']
bike_station_df

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.ebikes,extra.has_ebikes,extra.last_updated,extra.normal_bikes,extra.renting,extra.returning,extra.slots,extra.uid,lat_long
0,13,21,7a19c49f486d7c0c02b3685d7b240448,49.262487,-123.114397,10th & Cambie,2023-07-04T16:25:07.793000Z,4,True,1688487668,17,True,True,35,0001,"49.262487,-123.114397"
1,1,15,32603a87cfca71d0f7dfa3513bad69d5,49.274566,-123.121817,Yaletown-Roundhouse Station,2023-07-04T16:25:07.790000Z,1,True,1688487886,14,True,True,16,0004,"49.274566,-123.121817"
2,14,11,6d42fa40360f9a6b2bf641c7b8bb2862,49.279764,-123.110154,Dunsmuir & Beatty,2023-07-04T16:25:07.790000Z,1,True,1688487574,10,True,True,26,0005,"49.279764,-123.110154"
3,9,6,66f873d641d448bd1572ab086665a458,49.260599,-123.113504,12th & Yukon (City Hall),2023-07-04T16:25:07.792000Z,5,True,1688487817,1,True,True,16,0007,"49.260599,-123.113504"
4,5,10,485d4d24c803cfde829ab89699fed833,49.264215,-123.117772,8th & Ash,2023-07-04T16:25:07.799000Z,1,True,1688487829,9,True,True,16,0008,"49.264215,-123.117772"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,23,0,cc2084e365f6f230fe26bae0b6867775,49.277178,-123.130000,Hornby & Drake,2023-07-04T16:25:07.718000Z,0,True,1688487801,0,True,True,24,0196,"49.277178,-123.13"
239,6,10,f38bac9712f3beee2149ba2a97048878,49.291597,-123.129158,Cardero & Bayshore,2023-07-04T16:25:07.660000Z,1,True,1688487545,9,True,True,20,0200,"49.291597,-123.129158"
240,22,0,9ee3bc9e836768e3e04e4f3328dae03b,49.247204,-123.101549,27th & Main,2023-07-04T16:25:07.835000Z,0,True,1688487783,0,True,True,22,0565,"49.247204,-123.101549"
241,13,4,aa1cbf97abc3cd50515bc54633a9bb2a,49.285670,-123.112543,Cordova & Granville,2023-07-04T16:25:07.841000Z,2,True,1688487889,2,True,True,18,0021,"49.2856698,-123.1125432"


In [6]:
#create new variable locations
locations = bike_station_df.lat_long.tolist()


In [7]:
#number of total bikes available at each station
final_bike_station_df = bike_station_df
final_bike_station_df = final_bike_station_df.drop(['id','timestamp','extra.ebikes','extra.has_ebikes', 'extra.last_updated',
                                   'extra.normal_bikes','extra.renting','extra.returning','extra.slots','extra.uid'],axis=1)
final_bike_station_df

,empty_slots,free_bikes,latitude,longitude,name,lat_long
0,13,21,49.262487,-123.114397,10th & Cambie,"49.262487,-123.114397"
1,1,15,49.274566,-123.121817,Yaletown-Roundhouse Station,"49.274566,-123.121817"
2,14,11,49.279764,-123.110154,Dunsmuir & Beatty,"49.279764,-123.110154"
3,9,6,49.260599,-123.113504,12th & Yukon (City Hall),"49.260599,-123.113504"
4,5,10,49.264215,-123.117772,8th & Ash,"49.264215,-123.117772"
...,...,...,...,...,...,...
238,23,0,49.277178,-123.130000,Hornby & Drake,"49.277178,-123.13"
239,6,10,49.291597,-123.129158,Cardero & Bayshore,"49.291597,-123.129158"
240,22,0,49.247204,-123.101549,27th & Main,"49.247204,-123.101549"
241,13,4,49.285670,-123.112543,Cordova & Granville,"49.2856698,-123.1125432"


In [8]:
final_bike_station_df.rename(columns={'latitude': 'Latitude'}, inplace=True)
final_bike_station_df.rename(columns={'longitude': 'Longitude'}, inplace=True)


In [9]:
final_bike_station_df.to_csv('bike_station.csv', index=False)
print("CSV file saved successfully.")

CSV file saved successfully.


In [12]:
# check for duplicate rows
duplicate_rows = final_bike_station_df.duplicated()

# Count the number of duplicate rows
num_duplicates = duplicate_rows.sum()

# Display the duplicate rows
duplicate_data = final_bike_station_df[duplicate_rows]
print("Number of duplicate rows:", num_duplicates)
print("Duplicate rows:")
print(duplicate_data)

Number of duplicate rows: 0
Duplicate rows:
Empty DataFrame
Columns: [empty_slots, free_bikes, Latitude, Longitude, name, lat_long]
Index: []
